# ATS-Optimized Resume Generator

This notebook provides an end-to-end automated solution for generating ATS-optimized resumes tailored to specific job descriptions. The system will:

1. Extract job details from a provided URL
2. Analyze the job requirements using AI
3. Load your existing resume
4. Generate a tailored version optimized for ATS systems
5. Save the result as both PDF and DOCX files

## Features

- **ATS Optimization**: Ensures resume passes automated tracking systems
- **Keyword Matching**: Intelligently incorporates relevant keywords from job descriptions
- **Format Optimization**: Creates clean, parsable documents in multiple formats
- **Iterative Improvement**: Uses validation to continuously improve the resume
- **Quality Control**: Scores resumes and provides feedback on areas for improvement

## Setup and Dependencies

First, let's install the required packages:

In [ ]:
!pip install --quiet markdownify beautifulsoup4 pydantic html-to-markdown langchain_community pymupdf4llm pdfkit
!apt-get install -y wkhtmltopdf
!pip install --quiet jinja2 pdfkit md2docx-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libudev1 libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 

## Import Libraries

Now let's import all the necessary libraries:

In [ ]:
import os
import time
import tempfile
import pdfkit
import pymupdf4llm
from urllib.request import urlopen
from bs4 import BeautifulSoup
from google import genai
from google.colab import userdata
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import MarkdownifyTransformer
from IPython.display import Markdown as mrkdwn
from pydantic import BaseModel
from md2docx_python.src.md2docx_python import markdown_to_word

## Configuration

Define the configuration settings for the resume generator:

In [ ]:
# Configuration settings
CONFIG = {
    "output_dir": "/content/drive/MyDrive/resumes/",  # Change this to your preferred output location
    "resume_pdf_path": "/content/drive/MyDrive/Ajay Jayaraj.pdf",  # Change this to your resume path
    "api_key_name": "GOOGLE_API_KEY_1",  # Name of your API key in Google Colab secrets
    "model_name": "gemini-2.0-flash",  # Model for resume generation
    "validation_model_name": "gemini-2.0-flash",  # Model for validation
    "extract_model_name": "gemini-2.0-flash",  # Model for extraction
    "min_score": 80,  # Minimum score required for the resume
    "max_iterations": 10,  # Maximum number of iterations for improving the resume
}

## Helper Functions

Let's define helper functions for different parts of the resume generation process.

### API Client Setup

Function to set up the Google API client:

In [ ]:
def setup_api_client():
    """Set up and return the Google API client using the stored key."""
    try:
        api_key = userdata.get(CONFIG["api_key_name"])
        if not api_key:
            raise ValueError(f"API key '{CONFIG['api_key_name']}' not found in userdata")
        return genai.Client(api_key=api_key)
    except Exception as e:
        print(f"Error setting up API client: {e}")
        return None

### Job Description Processing

Function to fetch and process job descriptions:

In [ ]:
def fetch_job_description(url):
    """Fetch and convert job description from URL to markdown."""
    try:
        print(f"Fetching job description from: {url}")
        urls = [url]
        loader = AsyncHtmlLoader(urls)
        docs = loader.load()

        md = MarkdownifyTransformer()
        job_desc_extract = md.transform_documents(docs)

        print(f"Successfully extracted job description ({len(str(job_desc_extract))} characters)")
        return str(job_desc_extract)
    except Exception as e:
        print(f"Error fetching job description: {e}")
        return None

### Job Details Extraction

Function to extract relevant details from the job description:

In [ ]:
def extract_job_details(client, job_description_text):
    """Extract relevant details from job description for resume targeting."""
    try:
        extraction_prompt = f"""
        You are an expert ATS (Applicant Tracking System) analyst and job description interpreter.
        Your task is to extract critical information from job descriptions that will help a candidate's resume pass ATS scanners and impress human recruiters.

        **Objective:**
        Perform a comprehensive analysis of the provided job posting to identify:

        1. **Company Information:**
           - Company name
           - Industry sector
           - Company size (if mentioned)
           - Company culture indicators

        2. **Key Technical Requirements:**
           - Required technical skills (technologies, programming languages, platforms, tools)
           - Experience levels for each skill (beginner/intermediate/expert/years required)
           - Specific technical certifications

        3. **ATS Keywords:**
           - Primary hard skills keywords (frequency > 2 mentions)
           - Secondary hard skills keywords (mentioned 1-2 times)
           - Soft skills and traits emphasized
           - Domain/industry-specific terminology

        4. **Experience Requirements:**
           - Total years of experience required
           - Specific role experience requirements
           - Industry experience preferences

        5. **Education and Certifications:**
           - Minimum educational requirements
           - Preferred educational background
           - Required or preferred certifications

        6. **Role Responsibilities:**
           - Primary job functions (ranked by prominence in the description)
           - Key deliverables or performance expectations
           - Team/cross-functional collaboration requirements

        7. **Project or Domain Knowledge:**
           - Specific projects/product types mentioned
           - Domain expertise requirements
           - Industry-specific knowledge areas

        8. **Formatted Output:**
           - Present your findings in clearly labeled sections
           - For each skill/keyword, include its prominence/frequency in the job posting
           - Highlight "must-have" vs. "nice-to-have" requirements when distinguishable

        This analysis should enable a candidate to:
        - Prioritize which skills and experiences to emphasize
        - Ensure all critical ATS keywords are included
        - Understand what aspects of their background to highlight
        - Tailor their resume's language to match the employer's terminology

        Here is the job description to analyze:
        ```
        {job_description_text}
        ```
        """

        response = client.models.generate_content(
            model=CONFIG["extract_model_name"],
            contents=extraction_prompt,
        )

        return response.text
    except Exception as e:
        print(f"Error extracting job details: {e}")
        return None

### Resume Loading

Function to load the existing resume content:

In [ ]:
def load_resume_content(resume_path):
    """Load and return the content of the existing resume in markdown format."""
    try:
        print(f"Loading resume from: {resume_path}")
        resume_text = pymupdf4llm.to_markdown(resume_path)
        return resume_text
    except Exception as e:
        print(f"Error loading resume: {e}")
        return None

### Resume Generation

Function to generate a tailored resume based on job details:

In [ ]:
def generate_tailored_resume(client, job_details, resume_text, feedback=0, potential_errors=None):
    """Generate a resume tailored to the job description."""
    try:
        generation_prompt = f"""
        # RESUME TAILORING SYSTEM: ATS OPTIMIZATION SPECIALIST

        ## SYSTEM ROLE
        You are ResumeAI, an advanced resume optimization system with expert knowledge in:
        - ATS (Applicant Tracking System) algorithms and scoring
        - Industry-specific keyword optimization
        - Data-driven resume formatting
        - Recruiter psychology and decision patterns
        - Technical role requirements across industries

        ## INPUT DATA
        ### JOB ANALYSIS:
        ```
        {job_details}
        ```

        ### CURRENT RESUME:
        ```
        {resume_text}
        ```

        ## TASK DESCRIPTION
        Create a perfectly tailored resume that will:
        1. Score in the top 10% of all ATS systems (Lever, Greenhouse, Workday, Taleo)
        2. Pass automated keyword scanning with >90% match rate
        3. Appeal to human recruiters' psychology and scanning patterns
        4. Position the candidate as an ideal match for the role

        ## OPTIMIZATION REQUIREMENTS

        ### ATS COMPATIBILITY (CRITICAL)
        - Use standard section headings: "Professional Summary", "Skills", "Experience", "Education", "Certifications"
        - Include a clean single-column format with no tables, columns, headers/footers or graphics
        - Ensure all job titles, companies, dates are in consistent, parsable formats
        - Maintain proper hierarchical structure (h1, h2, h3, etc.)
        - Incorporate exact keywords from job description naturally into the content
        - Use standard bullet point formatting for experience points

        ### CONTENT OPTIMIZATION (CRITICAL)
        - Place highest-value keywords in the first 3-5 words of bullet points
        - Begin each bullet with strong action verbs in past tense (except current role)
        - Include metrics and quantifiable achievements in 70% of experience bullets
        - Maintain keyword density of 5-7% for primary job requirements
        - Eliminate personal pronouns entirely (no "I", "my", "we")
        - Match 85%+ of the "required skills" from the job description
        - Include industry-specific terminology that demonstrates insider knowledge

        ### RECRUITER PSYCHOLOGY (HIGH PRIORITY)
        - Front-load accomplishments that directly address job requirements
        - Use recognized industry terms that signal relevance
        - Include specific tools, platforms, methodologies mentioned in job description
        - Format for the 6-second scan (strategic bold on key terms)
        - Include recognizable certifications, tools and methodologies
        - Mirror the language style of the job description (formal/technical/collaborative)

        ### FORMATTING STANDARDS (MEDIUM PRIORITY)
        - Use bullet points of 1-2 lines maximum (never paragraphs)
        - Maintain consistent date formatting (MM/YYYY — MM/YYYY)
        - Use standard fonts and spacing
        - Ensure the resume is 1-2 pages maximum
        - Use markdown formatting for the final output

        ## RESPONSE FORMAT
        Respond in the schema:

        ```
        {{
          "generated_resume": "full markdown resume content here",
          "company_name": "company name extracted from job description"
        }}
        ```

        ### QUALITY CHECKS
        - Ensure proper grammar, spelling and punctuation
        - Verify all dates and formatting are consistent
        - Check for proper keyword inclusion and density
        - Score: {feedback}
        - Previous feedback: {potential_errors}
        """

        class generation_schema(BaseModel):
            generated_resume: str
            company_name: str

        response = client.chats.create(model=CONFIG["model_name"])
        result = response.send_message(
            generation_prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": generation_schema.model_json_schema(),
            }
        )

        return result.parsed
    except Exception as e:
        print(f"Error generating tailored resume: {e}")
        return None

### Resume Validation

Function to validate the generated resume and provide feedback:

In [ ]:
def validate_resume(client, job_details, generated_resume_text):
    """Validate the generated resume and provide feedback."""
    try:
        validation_prompt = f"""
        # RESUME EVALUATION SYSTEM: ATS AND RECRUITER ASSESSMENT

        ## SYSTEM ROLE
        You are ResumeEval, an expert resume evaluation system that analyzes resumes through the lens of:
        - ATS scoring algorithms
        - Recruiter attention patterns
        - Keyword optimization
        - Content effectiveness
        - Formatting standards

        ## INPUT DATA
        ### JOB ANALYSIS:
        ```
        {job_details}
        ```

        ### GENERATED RESUME:
        ```
        {generated_resume_text}
        ```

        ## EVALUATION TASK
        Perform a comprehensive analysis of the resume against the job requirements and provide:
        1. Numerical scoring across key dimensions
        2. Detailed feedback on strengths and weaknesses
        3. Specific improvement suggestions

        ## EVALUATION CRITERIA

        ### ATS COMPATIBILITY (40% of score)
        - Presence of standard section headers
        - Keyword match percentage with job description
        - Proper formatting for parsing
        - Consistent structure and organization
        - Absence of ATS-blocking elements

        ### CONTENT EFFECTIVENESS (30% of score)
        - Relevance of experience to job requirements
        - Strength of accomplishment statements
        - Presence of metrics and quantifiable results
        - Skill representation and alignment
        - Overall narrative strength

        ### RECRUITER APPEAL (20% of score)
        - Scannability and clarity
        - Impactful first impression
        - Strategic emphasis on key qualifications
        - Differentiation factors
        - Professional tone and language

        ### TECHNICAL ACCURACY (10% of score)
        - Grammar and spelling
        - Consistency in formatting
        - Appropriate length
        - Logical organization
        - Professional presentation

        ## RESPONSE FORMAT
        Respond in the schema:

        ```
        {{
          "validation_score": 0-100,
          "feedbacks": ["specific feedback point 1", "specific feedback point 2", ...],
          "potential_errors": ["error or weakness 1", "error or weakness 2", ...],
          "company_name": "company name extracted from job description"
        }}
        ```

        The validation score should be honest but fair, reflecting how well the resume would perform in a real application scenario. Provide at least 3 substantive feedback points and identify any potential weaknesses.
        """

        class validation_schema(BaseModel):
            validation_score: int
            feedbacks: list[str]
            potential_errors: list[str]
            company_name: str

        response = client.chats.create(model=CONFIG["validation_model_name"])
        result = response.send_message(
            validation_prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": validation_schema.model_json_schema(),
            }
        )

        return result.parsed
    except Exception as e:
        print(f"Error validating resume: {e}")
        return None

### Cover Letter Generation

In [ ]:
def generate_cover_letter(client, job_details, resume_text, company_name):
    """Generate a cover letter tailored to the job description."""
    try:
        cover_letter_prompt = f"""
        # COVER LETTER GENERATION SYSTEM: ATS-OPTIMIZED PROFESSIONAL LETTER

        ## SYSTEM ROLE
        You are CoverLetterAI, an advanced cover letter generation system with expert knowledge in:
        - Professional business letter writing
        - ATS keyword optimization
        - Personal brand positioning
        - Job-specific value proposition articulation
        - Recruiter psychology and decision patterns

        ## INPUT DATA
        ### JOB ANALYSIS:
        ```
        {job_details}
        ```

        ### CANDIDATE RESUME:
        ```
        {resume_text}
        ```

        ## TASK DESCRIPTION
        Create a professionally formatted, ATS-optimized cover letter that:
        1. Addresses the specific needs and requirements of the position
        2. Highlights the candidate's most relevant experiences and skills
        3. Demonstrates clear understanding of the company and role
        4. Includes appropriate keywords for ATS optimization
        5. Presents a compelling case for the candidate's fit
        6. Add Candidate information accordingly.

        ## COVER LETTER REQUIREMENTS

        ### STRUCTURE (CRITICAL)
        - Include proper business letter formatting (date, address, salutation, etc.)
        - Present information in 3-4 concise paragraphs:
          1. Introduction with position interest and brief value proposition
          2. Main body highlighting 2-3 key qualifications aligned with job requirements
          3. Additional paragraph on soft skills or cultural fit (optional)
          4. Closing paragraph with call to action and contact information
        - Total length: 250-350 words maximum
        - Addressed to "Hiring Manager" if no specific name is available

        ### CONTENT OPTIMIZATION (CRITICAL)
        - Incorporate 5-7 exact-match keywords from the job description
        - Highlight top 2-3 achievements relevant to the position
        - Include technical skills mentioned in the job posting
        - Demonstrate knowledge of the company and industry
        - Match tone and language to company culture
        - Balance confidence with humility
        - Show enthusiasm for the specific role

        ### ATS COMPATIBILITY
        - Use standard formatting (no columns, tables, or graphics)
        - Include role title and company name in the opening
        - Ensure proper spacing and structure
        - Use industry-standard terminology
        - Keep content simple and scannable

        ## RESPONSE FORMAT
        Respond in the schema:

        ```
        {{
          "cover_letter": "full markdown cover letter content here",
          "company_name": "{company_name}"
        }}
        ```

        Include proper markdown formatting for the business letter structure. The cover letter should be ready for PDF conversion with minimal editing.
        """

        class cover_letter_schema(BaseModel):
            cover_letter: str
            company_name: str

        response = client.chats.create(model=CONFIG["model_name"])
        result = response.send_message(
            cover_letter_prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": cover_letter_schema.model_json_schema(),
            }
        )
        # print(result.parsed)
        return result.parsed
    except Exception as e:
        print(f"Error generating cover letter: {e}")
        return None


def create_html_cover_letter(client, cover_letter_text, company_name):
    """Convert the markdown cover letter to HTML with proper styling."""
    try:
        html_prompt = f"""
        # COVER LETTER HTML CONVERTER: ATS-OPTIMIZED DESIGN

        ## SYSTEM ROLE
        You are CoverLetterHTML, an expert system that converts markdown cover letters to ATS-compatible, professionally styled HTML documents optimized for both PDF conversion and ATS parsing.

        ## INPUT DATA
        ### MARKDOWN COVER LETTER:
        ```
        {cover_letter_text}
        ```

        ## CONVERSION REQUIREMENTS

        ### ATS COMPATIBILITY (CRITICAL)
        - Create a simple layout without tables, complex divs, or complex CSS
        - Use semantic HTML5 elements appropriately
        - Ensure all text content is selectable and not in images
        - Avoid background images, background colors on text containers, or fancy styling
        - Use inline CSS only (no external stylesheets or <style> blocks)

        ### FORMATTING STANDARDS (HIGH PRIORITY)
        - Set page size for A4 paper (210mm × 297mm)
        - Use professional, system-standard fonts (Arial, Calibri, Helvetica)
        - Set appropriate margins (25mm all sides)
        - Use proper line height (1.15-1.5) for readability
        - Ensure proper spacing between paragraphs
        - Maintain business letter format standards

        ### VISUAL PRESENTATION (MEDIUM PRIORITY)
        - Use subtle styling for header elements (e.g., date, addresses)
        - Keep the design clean and professional
        - Ensure sufficient contrast for all text
        - Create proper paragraph spacing

        ### PRINT OPTIMIZATION
        - Set proper page margins for printing
        - Ensure all content fits on a single A4 page
        - Optimize spacing for professional appearance

        ## RESPONSE FORMAT
        Respond with valid HTML that has all CSS inline (not in a style tag). The HTML should:

        1. Be complete with <!DOCTYPE html>, <html>, <head>, and <body> tags
        2. Include appropriate meta tags for encoding and viewport
        3. Include a title tag with "{company_name} - Cover Letter"
        4. Contain all necessary inline styles
        5. Maintain the exact content from the markdown, but in optimized HTML format

        Response schema:
        ```
        {{
          "html": "<!DOCTYPE html><html>...</html>",
          "company_name": "{company_name}"
        }}
        ```
        """

        class html_schema(BaseModel):
            html: str
            company_name: str

        response = client.models.generate_content(
            model=CONFIG["extract_model_name"],
            contents=html_prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": html_schema.model_json_schema(),
            }
        )

        return response.parsed
    except Exception as e:
        print(f"Error creating HTML cover letter: {e}")
        return None

### HTML Generation

Function to convert the markdown resume to HTML with proper styling:

In [ ]:
def create_html_resume(client, generated_resume_text, company_name):
    """Convert the markdown resume to HTML with proper styling."""
    try:
        html_prompt = f"""
        # RESUME HTML CONVERTER: ATS-OPTIMIZED DESIGN SYSTEM

        ## SYSTEM ROLE
        You are ResumeHTML, an expert system that converts markdown resumes to ATS-compatible, professionally styled HTML documents optimized for both PDF conversion and ATS parsing.

        ## INPUT DATA
        ### MARKDOWN RESUME:
        ```
        {generated_resume_text}
        ```

        ## CONVERSION REQUIREMENTS

        ### ATS COMPATIBILITY (CRITICAL)
        - Create a single-column layout without tables, divs that mimic columns, or complex CSS
        - Use semantic HTML5 elements appropriately (header, section, h1-h6, ul, li, p)
        - Ensure all text content is selectable and not in images
        - Avoid background images, background colors on text containers, or fancy styling
        - Use inline CSS only (no external stylesheets or <style> blocks)

        ### FORMATTING STANDARDS (HIGH PRIORITY)
        - Set page size for A4 paper (210mm × 297mm)
        - Use professional, system-standard fonts (Arial, Calibri, Helvetica)
        - Set appropriate margins (10-15mm all sides)
        - Use proper line height (1.15-1.5) for readability
        - Ensure proper spacing between sections
        - Create subtle visual hierarchy with minimal styling

        ### VISUAL ENHANCEMENTS (MEDIUM PRIORITY)
        - Use subtle borders or horizontal rules to separate sections
        - Apply minimal color for section headings only (navy blue or dark gray)
        - Bold company names and job titles
        - Ensure sufficient contrast ratios for all text
        - Create clean, consistent indentation for list items

        ### PRINT OPTIMIZATION
        - Set proper page margins for printing
        - Ensure all content fits on standard A4 page(s)
        - Optimize spacing to prevent awkward page breaks
        - Add minimal page margin

        ## RESPONSE FORMAT
        Respond with valid HTML that has all CSS inline (not in a style tag). The HTML should:

        1. Be complete with <!DOCTYPE html>, <html>, <head>, and <body> tags
        2. Include appropriate meta tags for encoding and viewport
        3. Include a title tag with "{company_name} - Tailored Resume"
        4. Contain all necessary inline styles
        5. Maintain the exact content from the markdown, but in optimized HTML format

        Response schema:
        ```
        {{
          "html": "<!DOCTYPE html><html>...</html>",
          "company_name": "{company_name}"
        }}
        ```
        """

        class html_schema(BaseModel):
            html: str
            company_name: str

        response = client.models.generate_content(
            model=CONFIG["extract_model_name"],
            contents=html_prompt,
            config={
                "response_mime_type": "application/json",
                "response_schema": html_schema.model_json_schema(),
            }
        )

        return response.parsed
    except Exception as e:
        print(f"Error creating HTML resume: {e}")
        return None

### File Saving

Function to save the resume to PDF and DOCX files:

In [ ]:
def save_resume_to_files(generated_resume_text, html_content, company_name):
    """Save the resume to PDF and DOCX files in a company-specific dated folder."""
    try:
        # Get current date for folder naming
        from datetime import datetime
        current_date = datetime.now().strftime("%Y_%m_%d")

        # Clean company name for folder naming
        clean_company_name = ''.join(c if c.isalnum() else '_' for c in company_name)

        # Create company-specific folder with date
        company_folder = f"{clean_company_name}_{current_date}"
        company_folder_path = os.path.join(CONFIG["output_dir"], company_folder)
        os.makedirs(company_folder_path, exist_ok=True)

        # Define output paths
        pdf_path = os.path.join(company_folder_path, "ajay_jayaraj_resume.pdf")
        docx_path = os.path.join(company_folder_path, "ajay_jayaraj_resume.docx")

        # Save as PDF
        print(f"Saving PDF resume to: {pdf_path}")
        pdfkit.from_string(html_content, pdf_path)

        # Save as DOCX
        print(f"Saving DOCX resume to: {docx_path}")

        # Create a temporary markdown file
        with tempfile.NamedTemporaryFile(suffix='.md', mode='w', delete=False) as temp_md_file:
            temp_md_path = temp_md_file.name
            temp_md_file.write(generated_resume_text)

        # Convert markdown to DOCX
        markdown_to_word(temp_md_path, docx_path)

        # Clean up temporary file
        os.unlink(temp_md_path)

        print("Resume files saved successfully!")
        return pdf_path, docx_path, company_folder_path
    except Exception as e:
        print(f"Error saving resume files: {e}")
        return None, None, None


def save_cover_letter_to_files(cover_letter_text, html_content, company_name, company_folder_path=None):
    """Save the cover letter to PDF and DOCX files in a company-specific dated folder."""
    try:
        # If company folder path is not provided, create it
        if company_folder_path is None:
            # Get current date for folder naming
            from datetime import datetime
            current_date = datetime.now().strftime("%Y_%m_%d")

            # Clean company name for folder naming
            clean_company_name = ''.join(c if c.isalnum() else '_' for c in company_name)

            # Create company-specific folder with date
            company_folder = f"{clean_company_name}_{current_date}"
            company_folder_path = os.path.join(CONFIG["output_dir"], company_folder)
            os.makedirs(company_folder_path, exist_ok=True)

        # Define output paths
        pdf_path = os.path.join(company_folder_path, "ajay_jayaraj_cover_letter.pdf")
        docx_path = os.path.join(company_folder_path, "ajay_jayaraj_cover_letter.docx")

        # Save as PDF
        print(f"Saving PDF cover letter to: {pdf_path}")
        pdfkit.from_string(html_content, pdf_path)

        # Save as DOCX
        print(f"Saving DOCX cover letter to: {docx_path}")

        # Create a temporary markdown file
        with tempfile.NamedTemporaryFile(suffix='.md', mode='w', delete=False) as temp_md_file:
            temp_md_path = temp_md_file.name
            temp_md_file.write(cover_letter_text)

        # Convert markdown to DOCX
        markdown_to_word(temp_md_path, docx_path)

        # Clean up temporary file
        os.unlink(temp_md_path)

        print("Cover letter files saved successfully!")
        return pdf_path, docx_path
    except Exception as e:
        print(f"Error saving cover letter files: {e}")
        return None, None

## Main Function

Let's put it all together with the main function that orchestrates the entire process:

In [ ]:
def main():
    """Main function to run the end-to-end resume and cover letter generation process."""
    # Setup API client
    client = setup_api_client()
    if not client:
        print("Failed to set up API client. Exiting.")
        return

    # Get job URL from user
    job_url = input("Enter the job URL: ")

    # Fetch job description
    job_description_text = fetch_job_description(job_url)
    if not job_description_text:
        print("Failed to fetch job description. Exiting.")
        return

    # Extract job details
    job_details = extract_job_details(client, job_description_text)
    if not job_details:
        print("Failed to extract job details. Exiting.")
        return

    # Print extracted job details for user review
    print("\n--- Extracted Job Details ---")
    print(job_details[:500] + "..." if len(job_details) > 500 else job_details)
    print("----------------------------\n")

    # Load resume content
    resume_text = load_resume_content(CONFIG["resume_pdf_path"])
    if not resume_text:
        print("Failed to load resume content. Exiting.")
        return

    # Generate tailored resume with validation and improvement loop
    feedback = 0
    potential_errors = None
    company_name = None
    best_resume = None
    best_score = 0

    for iteration in range(1, CONFIG["max_iterations"] + 1):
        print(f"\n--- Iteration {iteration}/{CONFIG['max_iterations']} ---")

        # Generate resume
        generated_resume = generate_tailored_resume(
            client, job_details, resume_text, feedback, potential_errors
        )

        if not generated_resume:
            print("Failed to generate resume. Trying again...")
            time.sleep(2)
            continue

        generated_resume_text = generated_resume['generated_resume']
        company_name = generated_resume['company_name']

        # Validate resume
        validation = validate_resume(client, job_details, generated_resume_text)

        if not validation:
            print("Failed to validate resume. Using generated resume without validation.")
            best_resume = generated_resume_text
            company_name = generated_resume['company_name']
            break

        feedback = validation['validation_score']
        potential_errors = validation['potential_errors']

        print(f"Validation Score: {feedback}")
        print(f"Company Name: {company_name}")
        print(f"Potential Errors: {potential_errors}")

        # Keep track of the best resume
        if feedback > best_score:
            best_score = feedback
            best_resume = generated_resume_text

        # Break if we've reached the minimum score
        if feedback >= CONFIG["min_score"]:
            print(f"Achieved target score of {CONFIG['min_score']}. Proceeding with this resume.")
            break

        # Wait before next iteration to avoid rate limiting
        if iteration < CONFIG["max_iterations"]:
            print("Waiting before next iteration...")
            time.sleep(3)

    # Use the best resume we found
    if not best_resume:
        print("Failed to generate a satisfactory resume. Exiting.")
        return

    # Convert resume to HTML
    html_result = create_html_resume(client, best_resume, company_name)

    if not html_result:
        print("Failed to create HTML resume. Exiting.")
        return

    html_content = html_result['html']

    # Save resume files to company-specific folder
    pdf_path, docx_path, company_folder_path = save_resume_to_files(best_resume, html_content, company_name)

    if pdf_path and docx_path:
        print(f"\nResume generation complete!")
        print(f"PDF saved to: {pdf_path}")
        print(f"DOCX saved to: {docx_path}")
    else:
        print("Failed to save resume files.")
        return

    # Generate cover letter
    print("\n--- Generating Cover Letter ---")
    cover_letter_result = generate_cover_letter(client, job_details, resume_text, company_name)

    if not cover_letter_result:
        print("Failed to generate cover letter. Exiting.")
        return

    cover_letter_text = cover_letter_result['cover_letter']
    # print(cover_letter_text)

    # Convert cover letter to HTML
    html_cover_letter_result = create_html_cover_letter(client, cover_letter_text, company_name)

    if not html_cover_letter_result:
        print("Failed to create HTML cover letter. Exiting.")
        return

    html_cover_letter_content = html_cover_letter_result['html']

    # Save cover letter files to the same company folder
    cover_letter_pdf_path, cover_letter_docx_path = save_cover_letter_to_files(
        cover_letter_text, html_cover_letter_content, company_name, company_folder_path
    )

    if cover_letter_pdf_path and cover_letter_docx_path:
        print(f"\nCover letter generation complete!")
        print(f"PDF saved to: {cover_letter_pdf_path}")
        print(f"DOCX saved to: {cover_letter_docx_path}")
        print(f"\nAll files saved in folder: {company_folder_path}")
    else:
        print("Failed to save cover letter files.")

## Execute the Resume Generator

Now let's run the resume generator:

In [ ]:
if __name__ == "__main__":
    main()

Enter the job URL: https://www.linkedin.com/jobs/view/4231580463
Fetching job description from: https://www.linkedin.com/jobs/view/4231580463


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.02s/it]


Successfully extracted job description (42028 characters)

--- Extracted Job Details ---
Okay, I will analyze the provided job description to extract the information needed to tailor a resume for the "Data Engineer, IN Data Engineering & Analytics" position at Amazon Business.

**1. Company Information:**

*   **Company Name:** Amazon Business
*   **Industry Sector:** Retail (as listed in the job details).  Likely also Technology, E-commerce.
*   **Company Size:** Over 200 applicants (as stated in the job posting) indicates a large and competitive environment.
*   **Company Culture ...
----------------------------

Loading resume from: /content/drive/MyDrive/Ajay Jayaraj.pdf

--- Iteration 1/10 ---
Validation Score: 78
Company Name: Amazon Business
Potential Errors: ['While the resume mentions AWS big data technologies (Hadoop, Hive, Spark, EMR) in the skills section, the experience section lacks concrete examples of using these technologies in projects. Providing specific instances of

## Conclusion

This notebook provides a comprehensive solution for generating ATS-optimized resumes tailored to specific job descriptions. The key improvements over the original code include:

1. **Enhanced ATS Optimization**: Better keyword extraction and placement
2. **Improved Prompts**: More detailed instructions for AI models
3. **Modular Structure**: Well-organized functions with proper error handling
4. **Iterative Improvement**: Resume validation and feedback loop
5. **Multiple Output Formats**: Both PDF and DOCX for maximum flexibility

### How to Use

1. Ensure your Google API key is stored in Google Colab secrets
2. Update the configuration settings with your resume path and output directory
3. Run the notebook
4. Enter the job URL when prompted
5. The system will generate and save your optimized resume in both PDF and DOCX formats

### Customization

You can customize the resume generation process by modifying the following:

- The `CONFIG` dictionary to change paths, model names, and other settings
- The prompt templates to adjust the style and content of the generated resume
- The validation criteria to focus on specific aspects of ATS optimization